<a href="https://colab.research.google.com/github/Rachel-0420/Cross-Entropy-Fast-Transient/blob/real-data/cross_entropy_btw_supernovae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
# import os
# !pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# !pip install astropy pzflow corner
# from pzflow import Flow
# import jax.numpy as jnp
# import pandas as pd
# import corner
# import os
# from astropy.table import Table
# from pzflow import Flow, FlowEnsemble
# # from pzflow.distributions import Uniform
# from pzflow.bijectors import Chain, StandardScaler, NeuralSplineCoupling, ColorTransform, InvSoftplus, RollingSplineCoupling, ShiftBounds
# from collections import namedtuple
# import scipy.stats as sps

In [2]:
# Paths for the Probability Cubes of some transient classes
# SNII and SNIbc are both supernovae and are similar. AGN is very different from the other two

data1 = "/Users/rachelouyang/Desktop/Research/24 Summer Research/ProbCube_Data/ProbCube_1229_1815__SNII-Templates.pkl"
data2 = "/Users/rachelouyang/Desktop/Research/24 Summer Research/ProbCube_Data/ProbCube_1229_0821__SNIbc-Templates.pkl"
data3 = "/Users/rachelouyang/Desktop/Research/24 Summer Research/ProbCube_Data/ProbCube_1229_1819__AGN.pkl"

In [5]:
with open(data1, 'rb') as f1:
    CoordinateDict1 = pickle.load(f1)
    Data1 = pickle.load(f1)

with open(data2, 'rb') as f2:
    CoordinateDict2 = pickle.load(f2)
    Data2 = pickle.load(f2)

with open(data3, 'rb') as f3:
    CoordinateDict3 = pickle.load(f3)
    Data3 = pickle.load(f3)

In [31]:
arr_2 = np.array(Data2, dtype=float)
shape = arr_2.shape
print(shape)

In [29]:
# Data2.astype(float)

In [30]:
#interpretation of the shape of Data

print(len(CoordinateDict2['BandPairs']))
print(len(CoordinateDict2['dT1s']))
print(len(CoordinateDict2['dT2s']))
print(len(CoordinateDict2['dT1s']) * len(CoordinateDict2['dT2s']))
print(len(CoordinateDict2['BinMag']))
print(len(CoordinateDict2['BinColor']))

28
65
67
4355
111
39


In [28]:
# Test of numpy sum function on high-D arrays

test_arr = np.array([[[1,2,3,4],[2,3,4,5],[3,3,3,3]],[[1,2,3,4],[2,3,4,5],[3,3,3,3]],[[1,2,3,4],[2,3,4,5],[3,3,3,3]],[[1,2,3,4],[2,3,4,5],[3,3,3,3]]])

total_1 = np.sum(test_arr)

total_2 = 0

for i in range(test_arr.shape[0]):
    for j in range(test_arr.shape[1]):
        for k in range(test_arr.shape[2]):
            total_2 += test_arr[i][j][k]

print(total_1 == total_2)

True


In [7]:
def normalize(data):

  total = np.sum(data)

  factor = 1 / total

  for i in range(shape[0]):
    for j in range(shape[1]):
      for k in range(shape[2]):
        for l in range(shape[3]):
          data[i][j][k][l] *= factor

  return None

In [8]:
normalize(Data1)
normalize(Data2)
normalize(Data3)

In [32]:
# CoordinateDict1

In [ ]:
count_nonzero1 = np.sum(Data1 != 0)
count_01 = np.sum(Data1 == 0)
print((count_01, count_nonzero1, count_01 + count_nonzero1))

count_nonzero3 = np.sum(Data3 != 0)
count_03 = np.sum(Data3 == 0)
print((count_03, count_nonzero3, count_03 + count_nonzero3))

(408921098, 12774022, 421695120)
(420485763, 1209357, 421695120)


In [ ]:
print(len(Data2))

28


In [9]:
part1 = Data2[:10]
part2 = Data2[10:20]
part3 = Data2[20:]

In [10]:
part1[np.where(part1 == 0.)] = 1.e-16

In [11]:
part2[np.where(part2 == 0.)] = 1.e-16

In [12]:
part3[np.where(part3 == 0.)] = 1.e-16

In [13]:
for i in range(len(part1)):
  Data2[i] = part1[i]
  Data2[10+i] = part2[i]

for j in range(len(part3)):
  Data2[20+j] = part3[j]

In [14]:
del part1
del part2
del part3

In [15]:
H1 = Data1 * np.log(Data2)

<ipython-input-15-f9dd387c4ab2>:1: RuntimeWarning: divide by zero encountered in log
  H1 = Data1 * np.log(Data2)
<ipython-input-15-f9dd387c4ab2>:1: RuntimeWarning: invalid value encountered in multiply
  H1 = Data1 * np.log(Data2)


In [28]:
del H1
del Data1
del Data2